In [1]:
import gc
import os
import sys
import h5py
import random
import json
import pickle
from pathlib import Path

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

package_path = Path(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))
sys.path.insert(0, str(package_path))

In [2]:
from src.neuronio.neuronio_data_utils import (
    DEFAULT_Y_SOMA_THRESHOLD,
    DEFAULT_Y_TRAIN_SOMA_BIAS,
    DEFAULT_Y_TRAIN_SOMA_SCALE,
    NEURONIO_DATA_DIM,
    NEURONIO_LABEL_DIM,
    NEURONIO_SIM_LEN,
    NEURONIO_SIM_PER_FILE,
    create_neuronio_input_type,
    parse_sim_experiment_file,
    get_data_files_from_folder, 
)

In [3]:
# General Config
general_config = dict()
general_config["seed"] = 0
general_config["device"] = 'cuda' if torch.cuda.is_available() else 'cpu'
general_config["train_valid_eval"] = False
general_config["artefacts_dir"]= "../models/new_exp/forget_True_rest_True_nummem_10"
torch_device = torch.device(general_config["device"])
print("Torch Device: ", torch_device)

Torch Device:  cuda


In [4]:
# Seeding & Determinism
os.environ['PYTHONHASHSEED'] = str(general_config["seed"])
random.seed(general_config["seed"])
np.random.seed(general_config["seed"])
torch.manual_seed(general_config["seed"])
torch.cuda.manual_seed(general_config["seed"])
torch.backends.cudnn.deterministic = True

In [5]:
data_dir_path = Path("D:/NeuronIO").expanduser().resolve() # TODO: change to neuronio data path
train_data_dir_path = data_dir_path / "train"  # TODO: change to train subfolder
test_data_dir_path = data_dir_path / "test/Data_test"  # TODO: change to test subfolder

data_config = dict()
train_data_dirs = [
    str(train_data_dir_path / "full_ergodic_train_batch_1"),
    str(train_data_dir_path / "full_ergodic_train_batch_2"),
    str(train_data_dir_path / "full_ergodic_train_batch_3"),
    str(train_data_dir_path / "full_ergodic_train_batch_4"),
    str(train_data_dir_path / "full_ergodic_train_batch_5"),
    str(train_data_dir_path / "full_ergodic_train_batch_6"),
    str(train_data_dir_path / "full_ergodic_train_batch_7"),
    str(train_data_dir_path / "full_ergodic_train_batch_8"),
    str(train_data_dir_path / "full_ergodic_train_batch_9"),
    str(train_data_dir_path / "full_ergodic_train_batch_10"),
]
test_data_dirs = [str(test_data_dir_path)]

data_config["train_data_dirs"] = train_data_dirs
data_config["test_data_dirs"] = test_data_dirs

data_config["data_dim"] = NEURONIO_DATA_DIM 
data_config["label_dim"] = NEURONIO_LABEL_DIM

train_files = get_data_files_from_folder(data_config["train_data_dirs"])
test_files = get_data_files_from_folder(data_config["test_data_dirs"])

## Load model

In [6]:
from src.expressive_leaky_memory_neuron import ELM
from src.expressive_leaky_memory_neuron_forget import ELMf
from src.neuronio.neuronio_data_utils import (
    NEURONIO_DATA_DIM, 
    NEURONIO_LABEL_DIM, 
    get_data_files_from_folder, 
    parse_sim_experiment_file,
    visualize_training_batch,
)
from src.neuronio.neuronio_data_loader_filtered import NeuronIO
from src.neuronio.neuronio_train_utils import NeuronioLoss
from src.neuronio.neuronio_eval_utils_filtered import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)
from src.neuronio.neuronio_data_utils import parse_sim_experiment_file, create_neuronio_input_type, DEFAULT_Y_SOMA_THRESHOLD
from src.neuronio.neuronio_viz_utils import visualize_neuron_workings

In [7]:
# load model config
with open(general_config["artefacts_dir"] + "/model_config.json") as f:
    model_config = json.load(f)

# load train config
with open(general_config["artefacts_dir"] + "/train_config.json") as f:
    train_config = json.load(f)

In [8]:
# Initialize the ELM model
if train_config['forget_gate']:
    model = ELMf(**model_config).to(torch_device)
else:
    model = ELM(**model_config).to(torch_device)


# Load the best model for evaluation
state_dict = torch.load(general_config["artefacts_dir"] + "/neuronio_best_model_state.pt", map_location=torch_device)
print(model.load_state_dict(state_dict))
# Visualize ELM model
print(model)

<All keys matched successfully>
ELMf(
  (mlp): RecursiveScriptModule(
    original_name=MLP
    (network): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=ReLU)
      (2): RecursiveScriptModule(original_name=Linear)
    )
  )
  (mlpf): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Linear)
    (1): RecursiveScriptModule(original_name=Sigmoid)
  )
  (w_y): RecursiveScriptModule(original_name=Linear)
)


C:\Users\Bingkun LIU\AppData\Local\Temp\ipykernel_32508\3546985411.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(general_config["artefacts_dir"

In [9]:
# apples to apples comparison
sample_file_name = "100523.p"
sample_sequence_index = 33
sample_select_start = 2000
sample_select_len = 2000

# load file
select_test_file = [file for file in test_files if sample_file_name in file][0]
viz_X, viz_y_spike, viz_y_soma = parse_sim_experiment_file(select_test_file)

# select sequence
viz_X = viz_X[:, sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]
viz_y_spike = viz_y_spike[sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]
viz_y_soma = viz_y_soma[sample_select_start: sample_select_start+sample_select_len, sample_sequence_index]

# preprocess sequence
synapse_types = create_neuronio_input_type()
viz_X = torch.tensor(viz_X.T * synapse_types[:]).to(torch_device)
viz_y_soma = np.clip(viz_y_soma, a_min=None, a_max=DEFAULT_Y_SOMA_THRESHOLD)

KeyboardInterrupt: 

In [26]:
visualize_neuron_workings(
    neuron=model,
    input_spikes=viz_X,
    target_spikes=viz_y_spike,
    target_soma=viz_y_soma,
    color_by_memory_tau=False
)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

## Evaluation with different burn-in

In [12]:
burn_in_times = np.arange(0, 151, 25)

In [13]:
# Gather all evaluation metrics
# set burn-in in
eval_results_full = {}
with torch.no_grad():
    for burn_in_time in burn_in_times:
        if general_config["train_valid_eval"]:
            random.seed(general_config["seed"])
            select_train_files = random.choices(train_files, k=10)
            train_predictions = compute_test_predictions_multiple_sim_files(
                neuron=model,
                test_files=select_train_files,
                input_window_size=train_config["input_window_size"],
                rest_start = train_config['rest_start'],
                device=torch_device,
            )
            train_results = filter_and_extract_core_results(*train_predictions, burn_in_times=burn_in_times, verbose=False)
    
            valid_predictions = compute_test_predictions_multiple_sim_files(
                neuron=model,
                test_files=valid_files,
                input_window_size=train_config["input_window_size"],
                rest_start = train_config['rest_start'],
                device=torch_device,
            )
            valid_results = filter_and_extract_core_results(*valid_predictions, burn_in_times=burn_in_times, verbose=False)
    
    
        test_predictions = compute_test_predictions_multiple_sim_files(
            neuron=model,
            test_files=test_files,
            burn_in_time=burn_in_time,
            input_window_size=train_config["input_window_size"],
            rest_start = True, #train_config['rest_start'],
            device=torch_device,
        )
        test_results = filter_and_extract_core_results(*test_predictions, verbose=False)
    
        eval_results = dict()
        if general_config["train_valid_eval"]:
            eval_results["train_results"] = train_results
            eval_results["valid_results"] = valid_results
        eval_results["test_results"] = test_results
        eval_results_full[burn_in_time] = eval_results

In [14]:
eval_results_full

{np.int64(0): {'test_results': {'TP @ 0.0025 FP': np.float64(0.020257826887661142),
   'AUC @ 0.0025 FP': np.float64(0.001448741559238797),
   'TP @ 0.0100 FP': np.float64(0.30033763044812767),
   'AUC @ 0.0100 FP': np.float64(0.12591620626151012),
   'AUC': np.float64(0.9443477803277374),
   'soma_explained_variance_percent': 56.355741532743544,
   'soma_RMSE': np.float64(2.025873361261238),
   'soma_MAE': 1.451216855160072}},
 np.int64(25): {'test_results': {'TP @ 0.0025 FP': np.float64(0.15224063842848373),
   'AUC @ 0.0025 FP': np.float64(0.07967771639042356),
   'TP @ 0.0100 FP': np.float64(0.39103744628606507),
   'AUC @ 0.0100 FP': np.float64(0.23579248525808733),
   'AUC': np.float64(0.9488942132092616),
   'soma_explained_variance_percent': 62.476497053999935,
   'soma_RMSE': np.float64(1.878322644566575),
   'soma_MAE': 1.3689848093914463}},
 np.int64(50): {'test_results': {'TP @ 0.0025 FP': np.float64(0.15669122160834867),
   'AUC @ 0.0025 FP': np.float64(0.08268569674647024

In [ ]:
# Show evaluation results
eval_results

## Preprocess starting points NeuronIO

In [1]:
ignore_time_from_start = 500
threshold = -65
threshold_h = -60
section_len = 200
start_point_persec = 20

In [12]:
save_path = '../data_processed/NeuronIOstartpoint/'
burn_in_times = np.arange(0, 151, 25)
for file_path in test_files:
    X, y_spike, y_soma = parse_sim_experiment_file(
        sim_experiment_file=file_path,
        include_params=False,
        )
    sim_len, sim_num = y_soma.shape
    num_sec = int(sim_len/section_len)
    sections = y_soma.T.reshape([sim_num,num_sec,section_len])
    start_time  = np.ones([sim_num, num_sec, start_point_persec], dtype=int)
    
    recover_points_sim = {k: [] for k in burn_in_times}
    
    for sim in range(sim_num):
        full_choice  = []
        for i in range(num_sec):
            choice = (sections[sim, i, :] < threshold).nonzero()[0] + i * section_len
            if len(choice) < start_point_persec:
                choice = (sections[sim, i, :] < threshold_h).nonzero()[0] + i * section_len

            full_choice.append(np.sort(choice))
            
            # try:
            #     start_time[sim, i, :] = np.random.choice(choice, start_point_persec, replace=False)
            # except:
            #     start_time[sim, i, :choice.shape[0]] = choice
            #     start_time[sim, i, choice.shape[0]:] = np.random.choice(choice, start_point_persec-choice.shape[0], replace=False)
                
        full_choice = np.hstack(full_choice)
        recover_points = {k: [0] for k in burn_in_times}
        burn_in = 150
        for t_next in full_choice:
            for burn_in in burn_in_times:
                if recover_points[burn_in][-1]>= 5500:
                    continue 
                if t_next - recover_points[burn_in][-1] > 500-burn_in:
                    recover_points[burn_in].append(t_last)                          
            t_last = t_next

        for burn_in in burn_in_times:
            recover_points_sim[burn_in].append(np.array(recover_points[burn_in]))
            
    #pad and stack with -1
    for burn_in in burn_in_times:
        max_length = max(len(arr) for arr in recover_points_sim[burn_in])
        # Pad each array with zeros to the maximum length
        padded = [np.pad(arr, (0, max_length - len(arr)), mode='constant', constant_values = -1) for arr in recover_points_sim[burn_in]]
        recover_points_sim[burn_in] = np.vstack(padded)
    
    #start_time = np.sort(start_time.reshape([sim_num, -1]), axis=1)
    #np.save(save_path + file_path[-92:-2]+'.npy', start_time, allow_pickle=True)
    with open(save_path + file_path[-92:-2]+'_recover.pkl', 'wb') as fp:
        pickle.dump(recover_points_sim, fp)

In [149]:
for sim in range(sim_num):
    print(y_soma[start_time[sim], sim].max())

-65.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.125
-60.15625
-65.1875
-65.0625
-65.0625
-65.0625
-60.40625
-65.125
-60.09375
-65.125
-65.0625
-65.375
-65.0625
-65.5
-65.0625
-60.0625
-65.0625
-65.1875
-65.0625
-65.0625
-65.0625
-60.03125
-65.375
-65.125
-60.5
-60.03125
-65.25
-60.03125
-60.0625
-65.3125
-65.0625
-60.03125
-60.0625
-60.125
-65.125
-65.0625
-65.0625
-65.0625
-60.0625
-60.0625
-65.0625
-65.0625
-65.0625
-65.0625
-65.1875
-65.9375
-65.0625
-65.0625
-60.28125
-65.0625
-65.0625
-65.0625
-60.15625
-65.0625
-65.0625
-65.125
-65.0625
-65.1875
-65.8125
-65.1875
-60.375
-60.03125
-65.125
-65.3125
-65.1875
-60.0625
-65.0625
-60.03125
-65.5
-65.0625
-65.0625
-60.9375
-60.15625
-65.125
-65.125
-60.0625
-60.625
-60.125
-60.5
-60.03125
-65.4375
-65.0625
-60.125
-60.125
-65.0625
-65.0625
-65.375
-65.1875
-65.0625
-60.03125
-69.5
-60.03125
-60.4375
-65.0625
-60.5
-65.125
-65.0625
-65.0625
-65.0625
-60.03125
-60.0625
-65.3125
-65.0625
-60.1875
-60.09375
-60.5
-65.0625
-60.96875
-60.531

In [13]:
save_path = '../data_processed/NeuronIOstartpoint/'
file_path = test_files[2]
start_time = np.load(save_path + file_path[-92:-2]+'.npy')
with open(save_path + file_path[-92:-2]+'_recover.pkl', 'rb') as fp:
    recover_points_sim = pickle.load(fp)

In [14]:
recover_points_sim[125]

array([[   0,  375,  750, ..., 5217, 5592,   -1],
       [   0,  375,  750, ..., 5162, 5537,   -1],
       [   0,  375,  714, ..., 4983, 5358, 5733],
       ...,
       [   0,  375,  750, ..., 5250, 5625,   -1],
       [   0,  316,  691, ..., 5182, 5557,   -1],
       [   0,  375,  676, ..., 5110, 5438, 5813]])

In [27]:
start_time

array([[  15,   17,   19, ..., 5962, 5979, 5989],
       [   3,   65,   69, ..., 5985, 5986, 5998],
       [   7,    8,   41, ..., 5981, 5991, 5998],
       ...,
       [   0,    2,    3, ..., 5939, 5985, 5987],
       [  10,   28,   42, ..., 5984, 5985, 5997],
       [   6,   12,   19, ..., 5968, 5981, 5993]])

In [34]:
np.diff(sec_ealiest, axis=1).max()

np.int64(379)

In [19]:
sort_start = np.sort(start_time, axis=1)

In [20]:
sort_start

array([[   0,    2,    9, ..., 5963, 5968, 5976],
       [   2,   14,   23, ..., 5932, 5934, 5936],
       [  10,   12,   31, ..., 5986, 5991, 5992],
       ...,
       [   8,   10,   21, ..., 5982, 5983, 5987],
       [   0,   25,   29, ..., 5981, 5988, 5993],
       [   4,   15,   23, ..., 5971, 5980, 5982]])

In [21]:
dif = np.diff(sort_start[:, np.arange(0, start_time.shape[1], start_point_persec)], axis=1)

In [23]:
dif.max()

np.int64(358)

In [6]:
for file_path in train_files:
    X, y_spike, y_soma = parse_sim_experiment_file(
        sim_experiment_file=file_path,
        include_params=False,
        )
    sim_len, sim_num = y_soma.shape
    #print(y_spike.sum(axis=0))
    print(np.sum(y_spike.sum(axis=0) > 10))
    #print(len(np.nonzero(y_spike.sum(axis=0) > 10)))
        

21
20
17
21
22
22
23
21
23
17
20
22
25
22
25
21
25
26
24
23
24
25
26
23
18
27
19
25
27
27
24
26
22
27
21
23
23
24
25
29
33
26
25
25
22
31
27
26
26
27
24
25
25
28
26
28
28
32
28
32
28
33
28
34
36
31
28
33
37
36
27
26
25
23
26
28
26
26
22
24
28


KeyboardInterrupt: 

In [19]:
X, y_spike, y_soma = parse_sim_experiment_file(
        sim_experiment_file=test_files[0],
        include_params=False,
        )
sim_len, sim_num = y_soma.shape
qualified = np.nonzero(y_spike.sum(axis=0) > 10)[0]
    #print(y_spike.sum(axis=0))
#print(np.sum(y_spike.sum(axis=0) > 10))
    #print(len(np.nonzero(y_spike.sum(axis=0) > 10)))

In [20]:
qualified

array([  0,   5,   9,  10,  11,  17,  21,  24,  27,  34,  40,  45,  54,
        66,  68,  72,  73,  76,  91,  97, 111, 114])

## Generating Random Input

## Generating Output from random input

In [ ]:
from src.neuronio.neuronio_eval_utils import (
    NeuronioEvaluator, 
    compute_test_predictions_multiple_sim_files, 
    filter_and_extract_core_results,
)

with torch.no_grad():
    test_predictions = compute_test_predictions_multiple_sim_files(
        neuron=model,
        test_files=test_files,
        burn_in_time=train_config["burn_in_time"],
        input_window_size=train_config["input_window_size"],
        device=torch_device,
    )
    test_results = filter_and_extract_core_results(*test_predictions, verbose=False)

eval_results = dict()
if general_config["train_valid_eval"]:
    eval_results["train_results"] = train_results
    eval_results["valid_results"] = valid_results
eval_results["test_results"] = test_results